<center style="font-size:48px;">Modeling</center>

<center>My attempt at utilizing machine learning to identify some initial data ppints to further invesigate</center>

# Import Libraries and Data

## Libraries

In [43]:
# Data Science
import pandas as pd
import joblib

# Machine Learning
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

## Data

In [44]:
# Location
locations = pd.read_csv('../CheckPoints/Locations_Clean.csv', parse_dates=['Timestamp'], index_col=0)
# Purchase Data
buys = pd.read_csv('../CheckPoints/Buys_Clean.csv', parse_dates=['timestamp'], index_col=0)

# Helper Functions

In [45]:
def findOutliers(model, df):
    preds = model.fit_predict(df)
    temp = pd.DataFrame(preds)
    if len(temp[0].unique()) == 1:
        print('No Outliers Detected. All points belong to same cluster.')
    else:
        outs = [ True if x == -1 else False for x in preds]
        display(df[outs])

# Machine Learning

## Location

Create a preprocessing pipeline. We need to drop the timestamp column as we already have the datetime parts seperated into their own variable. We can also drop the CarID variable as there is a one to one relationship with the CarID. We will need to encode the names variables as well as the job title and type.

In [46]:
locPreprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['Timestamp', 'id'] ),
        ('encoder', OneHotEncoder(), ['LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

We will use two algorithms that are implmented in scikit-learn; IsolationForest and LocalOutlierFactor. First lets look at IsolationForest.

In [47]:
# Model Pipeline
locIso = Pipeline(
    [
        ('preprocessor', locPreprocessor),
        ('model', IsolationForest())
    ]
)
# Find Outliers
findOutliers(locIso, locations)

No Outliers Detected. All points belong to same cluster.


It looks like this algorithm didnt find any outliers.

Now lets try LocalOutlierFactor.

In [48]:
# Model Pipeline
locLocalpipe = Pipeline(
    [
        ('preprocessor', locPreprocessor),
        ('model', LocalOutlierFactor(n_jobs = -1))
    ]
)
# Find Outliers
findOutliers(locLocalpipe, locations)

,FirstName,LastName,CurrentEmploymentType,CurrentEmploymentTitle,Timestamp,id,lat,long,day,hour,minute,Weekend
66601,Bertrand,Ovan,Facilities,Facilities Group Manager,2014-01-12 00:00:00,29.0,36.059926,24.893272,12.0,0.0,0.0,True
154320,Elsa,Orilla,Engineering,Drill Technician,2014-01-19 00:00:00,28.0,36.068748,24.882666,19.0,0.0,0.0,True
154321,Elsa,Orilla,Engineering,Drill Technician,2014-01-19 00:01:00,28.0,36.071066,24.879613,19.0,0.0,1.0,True
154322,Elsa,Orilla,Engineering,Drill Technician,2014-01-19 00:02:00,28.0,36.072861,24.876743,19.0,0.0,2.0,True
225301,Hennie,Osvaldo,Security,Perimeter Control,2014-01-14 03:43:00,21.0,36.060828,24.904150,14.0,3.0,43.0,False
330356,Isia,Vann,Security,Perimeter Control,2014-01-07 01:10:00,16.0,36.076832,24.865878,7.0,1.0,10.0,False
464669,Lucas,Alcazar,Information Technology,IT Helpdesk,2014-01-07 01:10:00,1.0,36.050386,24.879576,7.0,1.0,10.0,False
464670,Lucas,Alcazar,Information Technology,IT Helpdesk,2014-01-07 01:11:00,1.0,36.053128,24.882471,7.0,1.0,11.0,False
464671,Lucas,Alcazar,Information Technology,IT Helpdesk,2014-01-07 01:12:00,1.0,36.058610,24.882478,7.0,1.0,12.0,False
464672,Lucas,Alcazar,Information Technology,IT Helpdesk,2014-01-07 01:13:00,1.0,36.063921,24.882584,7.0,1.0,13.0,False


Awesome, our model gave us a start with 5 suspects to look at. Lucas Alcazar also had two different times pointed out.

## Purchases

Create a preprocessing pipeline. We need to drop the timestamp column as we already have the datetime parts seperated into their own variable. We can also drop the CarID variable as there is a one to one relationship with the CarID. Lets also drop the minute and seconda columns. With a chunk of the purchases not haveing this because they were batch reported it could alter results. We will need to encode the names, locations,  job title and type variables.

In [49]:
Buypreprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['timestamp',  'CarID', 'minute', 'second'] ),
        ('encoder', OneHotEncoder(), ['location', 'LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

Similar to above we will use two algorithms that are implmented in scikit-learn; IsolationForest and LocalOutlierFactor. First lets look at IsolationForest.

In [50]:
# Model Pipeline
BuyIso = Pipeline(
    [
        ('preprocessor', Buypreprocessor),
        ('model', IsolationForest())
    ]
)
# Find Outliers
findOutliers(BuyIso, buys)

No Outliers Detected. All points belong to same cluster.


Mhmmm, once again no outliers for this algorithm.

In [51]:
# Model Pipeline
BuyLocal = Pipeline(
    [
        ('preprocessor', Buypreprocessor),
        ('model', LocalOutlierFactor(n_jobs = -1))
    ]
)
# Find Outliers
findOutliers(BuyLocal, buys)

,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,Weekend,CarID,CurrentEmploymentType,CurrentEmploymentTitle
92,2014-01-06 15:39:00,Abila Airport,612.47,Henk,Mies,True,6,15,39,0,False,100.0,Facilities,Truck Driver
414,2014-01-09 09:54:00,Carlyle Chemical Inc.,529.47,Valeria,Morlun,True,9,9,54,0,False,100.0,Facilities,Truck Driver
479,2014-01-09 14:26:00,Carlyle Chemical Inc.,382.43,Valeria,Morlun,True,9,14,26,0,False,100.0,Facilities,Truck Driver
480,2014-01-09 14:28:00,Kronos Pipe and Irrigation,423.14,Irene,Nant,True,9,14,28,0,False,100.0,Facilities,Truck Driver
842,2014-01-13 19:20:00,Frydos Autosupply n' More,10000.00,Lucas,Alcazar,False,13,19,20,0,False,1.0,Information Technology,IT Helpdesk
898,2014-01-14 08:12:00,Hallowed Grounds,39.52,Kare,Orilla,False,14,8,12,0,False,27.0,Engineering,Drill Technician
906,2014-01-14 11:01:00,Stewart and Sons Fabrication,540.19,Benito,Hawelon,True,14,11,1,0,False,100.0,Facilities,Truck Driver
907,2014-01-14 11:45:00,Nationwide Refinery,570.95,Valeria,Morlun,True,14,11,45,0,False,100.0,Facilities,Truck Driver
1158,2014-01-16 09:52:00,Carlyle Chemical Inc.,419.93,Valeria,Morlun,True,16,9,52,0,False,100.0,Facilities,Truck Driver
1372,2014-01-18 12:03:00,Chostus Hotel,600.00,Sten,Sanjorge Jr.,True,18,12,3,0,True,31.0,Executive,President/CEO


Thats a decent list we can go through.

## Save our two models.

In [52]:
locLocalpipe.fit(locations)
joblib.dump(locLocalpipe, '../models/LocationLocalOutlierFactor.sav')
BuyLocal.fit(buys)
joblib.dump(BuyLocal, '../models/PurchaseLocalOutlierFactor.sav')

['../models/PurchaseLocalOutlierFactor.sav']

<div>
    <span  style="width:600px;display:inline-block;text-align:left">
        <a href="./CleanUp.ipynb">&#60;&#60;Data Cleanup</a>
    </span>
    <span style="width:600px;display:inline-block;text-align:right">
        <a href="./ObservationsQ1.ipynb">Observations Question 1&#62;&#62;</a>
    </span>
</div>
<div>
    <center>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="./Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="../README.md">Table of Contents</a>
        </span>
    </center>
</div>